In [38]:
val df = spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .option("subscribe", "stockData")
  .load()
df.printSchema

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



df: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]


In [39]:
val selectDF = df.select(get_json_object(($"value").cast("string"),"$.data.T").alias("ticker").cast("String"),
                         get_json_object(($"value").cast("string"),"$.data.p").alias("price").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.x").alias("exchnage_id").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.s").alias("trade_size").cast("Int"),
                         get_json_object(($"value").cast("string"),"$.data.t").alias("date_time").cast("Long"))


selectDF: org.apache.spark.sql.DataFrame = [ticker: string, price: int ... 3 more fields]


In [40]:
val temp = selectDF.na.drop()
val temp2 = temp.withColumn("date_time",(col("date_time")/1000000000).cast("timestamp"))
temp2.show

+------+-----+-----------+----------+--------------------+
|ticker|price|exchnage_id|trade_size|           date_time|
+------+-----+-----------+----------+--------------------+
|  TSLA|  990|          2|       100|2020-06-26 13:55:...|
|   SPY|  302|          3|        99|2020-06-26 15:21:...|
|   SPY|  302|          2|       100|2020-06-26 15:21:...|
|   SPY|  302|          2|         1|2020-06-26 15:21:...|
|   SPY|  302|          2|       400|2020-06-26 15:21:...|
|   SPY|  302|         17|         1|2020-06-26 15:21:...|
|   SPY|  302|         17|         1|2020-06-26 15:21:...|
|   SPY|  302|         17|      1300|2020-06-26 15:21:...|
|   SPY|  302|          2|         4|2020-06-26 15:21:...|
|   SPY|  302|          2|        20|2020-06-26 15:21:...|
|   SPY|  302|          2|         4|2020-06-26 15:21:...|
|   SPY|  302|         17|         1|2020-06-26 15:21:...|
|   SPY|  302|          9|       100|2020-06-26 15:21:...|
|   SPY|  302|          2|       400|2020-06-26 15:21:..

temp: org.apache.spark.sql.DataFrame = [ticker: string, price: int ... 3 more fields]
temp2: org.apache.spark.sql.DataFrame = [ticker: string, price: int ... 3 more fields]


In [48]:
val APPL = temp2.filter("ticker == 'AAPL'").sort(desc("date_time")).show

+------+-----+-----------+----------+--------------------+
|ticker|price|exchnage_id|trade_size|           date_time|
+------+-----+-----------+----------+--------------------+
|  AAPL|  358|         15|        52|2020-06-26 16:23:...|
|  AAPL|  358|         15|       200|2020-06-26 16:23:...|
|  AAPL|  358|         15|       200|2020-06-26 16:23:...|
|  AAPL|  358|          2|       100|2020-06-26 16:23:...|
|  AAPL|  358|          2|         2|2020-06-26 16:23:...|
|  AAPL|  358|          2|        12|2020-06-26 16:23:...|
|  AAPL|  358|         15|        25|2020-06-26 16:23:...|
|  AAPL|  358|         17|         2|2020-06-26 16:23:...|
|  AAPL|  358|         15|        52|2020-06-26 16:23:...|
|  AAPL|  358|         15|       100|2020-06-26 16:23:...|
|  AAPL|  358|         15|       100|2020-06-26 16:23:...|
|  AAPL|  358|         15|        51|2020-06-26 16:23:...|
|  AAPL|  358|         15|        50|2020-06-26 16:23:...|
|  AAPL|  358|         15|       300|2020-06-26 16:23:..

APPL: Unit = ()


In [49]:
val TSLA = temp2.filter("ticker == 'TSLA'").sort(desc("date_time")).show

+------+-----+-----------+----------+--------------------+
|ticker|price|exchnage_id|trade_size|           date_time|
+------+-----+-----------+----------+--------------------+
|  TSLA|  968|          3|         1|2020-06-26 16:23:...|
|  TSLA|  969|          2|        20|2020-06-26 16:23:...|
|  TSLA|  969|         17|        46|2020-06-26 16:23:...|
|  TSLA|  969|         17|        54|2020-06-26 16:23:...|
|  TSLA|  968|          2|        25|2020-06-26 16:22:...|
|  TSLA|  969|         17|         1|2020-06-26 16:22:...|
|  TSLA|  970|         15|       100|2020-06-26 16:22:...|
|  TSLA|  970|          3|         8|2020-06-26 16:21:...|
|  TSLA|  970|          3|         2|2020-06-26 16:21:...|
|  TSLA|  970|          3|         2|2020-06-26 16:21:...|
|  TSLA|  970|          3|         8|2020-06-26 16:21:...|
|  TSLA|  970|         17|         1|2020-06-26 16:21:...|
|  TSLA|  970|          3|        10|2020-06-26 16:21:...|
|  TSLA|  970|          3|        25|2020-06-26 16:21:..

TSLA: Unit = ()


In [50]:
val MSFT = temp2.filter("ticker == 'MSFT'").sort(desc("date_time")).show

+------+-----+-----------+----------+--------------------+
|ticker|price|exchnage_id|trade_size|           date_time|
+------+-----+-----------+----------+--------------------+
|  MSFT|  196|          2|       100|2020-06-26 16:23:...|
|  MSFT|  196|         17|         1|2020-06-26 16:23:...|
|  MSFT|  196|         17|         2|2020-06-26 16:23:...|
|  MSFT|  196|         17|         1|2020-06-26 16:23:...|
|  MSFT|  196|         17|         1|2020-06-26 16:23:...|
|  MSFT|  196|         17|         1|2020-06-26 16:23:...|
|  MSFT|  196|         17|        22|2020-06-26 16:23:...|
|  MSFT|  196|         15|       100|2020-06-26 16:23:...|
|  MSFT|  196|         15|       100|2020-06-26 16:23:...|
|  MSFT|  196|         15|        90|2020-06-26 16:23:...|
|  MSFT|  196|         15|       100|2020-06-26 16:23:...|
|  MSFT|  196|         15|        10|2020-06-26 16:23:...|
|  MSFT|  196|         15|       100|2020-06-26 16:23:...|
|  MSFT|  196|         17|         2|2020-06-26 16:22:..

MSFT: Unit = ()
